In [ ]:
#trying out Gemini's implementation of VAE



In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, input_dim=784, h_dim=200, z_dim=20):
        super(VAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, h_dim),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(h_dim, z_dim)
        self.fc_logvar = nn.Linear(h_dim, z_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, input_dim),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Flatten the image
        x = x.view(-1, 784)
        
        # Encode
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        
        # Sample latent vector
        z = self.reparameterize(mu, logvar)
        
        # Decode
        recon_x = self.decoder(z)
        return recon_x, mu, logvar

In [9]:
def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='mean')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [10]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim

# Hyperparameters
h_dim = 200
z_dim = 20
epochs = 10
batch_size = 64
learning_rate = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load MNIST dataset
transform = transforms.ToTensor()
mnist_data = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
data_loader = DataLoader(dataset=mnist_data, batch_size=batch_size, shuffle=True)

# Initialize model, optimizer
model = VAE(h_dim=h_dim, z_dim=z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    for i, (images, _) in enumerate(data_loader):
        # Flatten and move to device
        images = images.to(device)

        # Forward pass
        recon_images, mu, logvar = model(images)
        loss = vae_loss(recon_images, images, mu, logvar)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 400 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(data_loader)}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/938], Loss: 0.3249
Epoch [1/10], Step [200/938], Loss: 0.2732
Epoch [1/10], Step [300/938], Loss: 0.2710
Epoch [1/10], Step [400/938], Loss: 0.2739
Epoch [1/10], Step [500/938], Loss: 0.2730
Epoch [1/10], Step [600/938], Loss: 0.2777
Epoch [1/10], Step [700/938], Loss: 0.2582
Epoch [1/10], Step [800/938], Loss: 0.2672
Epoch [1/10], Step [900/938], Loss: 0.2554
Epoch [2/10], Step [100/938], Loss: 0.2689
Epoch [2/10], Step [200/938], Loss: 0.2570
Epoch [2/10], Step [300/938], Loss: 0.2528
Epoch [2/10], Step [400/938], Loss: 0.2663
Epoch [2/10], Step [500/938], Loss: 0.2627
Epoch [2/10], Step [600/938], Loss: 0.2531
Epoch [2/10], Step [700/938], Loss: 0.2647
Epoch [2/10], Step [800/938], Loss: 0.2629
Epoch [2/10], Step [900/938], Loss: 0.2690
Epoch [3/10], Step [100/938], Loss: 0.2650
Epoch [3/10], Step [200/938], Loss: 0.2726
Epoch [3/10], Step [300/938], Loss: 0.2676
Epoch [3/10], Step [400/938], Loss: 0.2601
Epoch [3/10], Step [500/938], Loss: 0.2791
Epoch [3/10

In [8]:
#sampling new images
import torch
import torch.nn as nn
from torchvision.utils import save_image
import os

# Define the VAE class (same as in the previous example)
class VAE(nn.Module):
    def __init__(self, input_dim=784, h_dim=200, z_dim=20):
        super(VAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, h_dim),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(h_dim, z_dim)
        self.fc_logvar = nn.Linear(h_dim, z_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, input_dim),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = x.view(-1, 784)
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decoder(z)
        return recon_x, mu, logvar

# --- Assumed: Model is trained and saved ---
# In a real scenario, you'd load a saved model state:
# model.load_state_dict(torch.load('vae_mnist.pth'))

# Example setup (replace with your trained model and device)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
z_dim = 20
#model = VAE(z_dim=z_dim).to(device)
# Note: This will not generate high-quality images without a trained model.
# Load your actual trained model state here.

# --- Sampling New Images ---
def generate_and_save_images(model, z_dim, num_samples=25):
    model.eval() # Set the model to evaluation mode
    with torch.no_grad():
        # 1. Sample from a standard normal distribution
        z = torch.randn(num_samples, z_dim).to(device)
        
        # 2. Pass samples through the decoder
        generated_images = model.decoder(z).cpu()
        
        # 3. Reshape and save
        generated_images = generated_images.view(num_samples, 1, 28, 28)
        
        # Create a directory to save images if it doesn't exist
        if not os.path.exists('generated_vae_images'):
            os.makedirs('generated_vae_images')
        
        # Save the images in a grid format
        save_image(generated_images, os.path.join('generated_vae_images', 'vae_samples.png'), nrow=5)
        print("Generated and saved new images to 'generated_vae_images/vae_samples.png'")

# Run the function to generate and save images
generate_and_save_images(model, z_dim)

Generated and saved new images to 'generated_vae_images/vae_samples.png'
